In [81]:
library(microbiome)
library(brms)
library(tidyverse)
library(broom)

In [47]:
data(dietswap)

In [48]:
otu <- otu_table(dietswap)
meta <- sample_data(dietswap)

In [52]:
head(meta)
rownames(otu) <- gsub(" ", "_", rownames(otu))
rownames(otu) <- gsub("\\.", "", rownames(otu))
rownames(otu) <- gsub("Clostridium_\\(sensu_stricto\\)", "Clostridium_sensu_stricto", rownames(otu))
otu.rel <- microbiome::transform(otu, "compositional")
head(otu.rel)

,subject,sex,nationality,group,sample,timepoint,timepoint.within.group,bmi_group
Sample-1,byn,Male,AAM,DI,Sample-1,4,1,obese
Sample-2,nms,Male,AFR,HE,Sample-2,2,1,lean
Sample-3,olt,Male,AFR,HE,Sample-3,2,1,overweight
Sample-4,pku,Female,AFR,HE,Sample-4,2,1,obese
Sample-5,qjy,Female,AFR,HE,Sample-5,2,1,overweight
Sample-6,riv,Female,AFR,HE,Sample-6,2,1,obese


,Sample-1,Sample-2,Sample-3,Sample-4,Sample-5,Sample-6,Sample-7,Sample-8,Sample-9,Sample-10,⋯,Sample-213,Sample-214,Sample-215,Sample-216,Sample-217,Sample-218,Sample-219,Sample-220,Sample-221,Sample-222
Actinomycetaceae,2.060778e-05,5.035640e-05,1.152133e-05,5.023935e-05,1.461157e-05,1.604835e-05,2.598353e-05,1.582669e-05,4.134911e-05,2.204031e-05,⋯,1.245257e-05,1.248259e-05,5.499382e-05,1.590784e-05,3.175146e-05,3.735251e-05,1.095725e-05,1.543964e-05,2.451799e-05,2.211021e-05
Aerococcus,1.873435e-06,7.515881e-07,5.486348e-07,1.196175e-06,9.132228e-07,8.024175e-07,1.082647e-06,7.193950e-07,1.315653e-05,9.183462e-07,⋯,7.325041e-07,8.321725e-07,1.896339e-06,1.060523e-06,7.559870e-07,1.436635e-06,9.131044e-07,8.577580e-07,1.167523e-06,7.370071e-07
Aeromonas,1.873435e-06,7.515881e-07,5.486348e-07,1.196175e-06,9.132228e-07,8.024175e-07,1.082647e-06,7.193950e-07,1.879505e-06,9.183462e-07,⋯,7.325041e-07,8.321725e-07,1.896339e-06,1.060523e-06,7.559870e-07,1.436635e-06,9.131044e-07,8.577580e-07,1.167523e-06,7.370071e-07
Akkermansia,2.186298e-03,4.604980e-03,2.323469e-03,1.935890e-02,1.215500e-03,8.264900e-04,1.774459e-03,1.350304e-03,2.230972e-03,7.266873e-03,⋯,1.812948e-03,8.879280e-04,2.267452e-02,4.256939e-03,1.468127e-03,1.116265e-03,8.829720e-04,1.208581e-03,5.522385e-03,2.096785e-03
Alcaligenes_faecalis_et_rel,1.686091e-04,9.470010e-05,1.031433e-04,1.495219e-04,9.771484e-05,1.107336e-04,1.342482e-04,8.273042e-05,1.898300e-04,4.022356e-04,⋯,1.040156e-04,1.040216e-04,1.636540e-03,5.854087e-04,5.314589e-04,5.085688e-04,3.104555e-04,9.263786e-05,1.587832e-04,1.466644e-04
Allistipes_et_rel,3.975803e-02,2.965767e-03,1.237720e-03,7.230879e-03,2.819119e-03,8.826593e-04,3.202470e-03,2.245232e-03,1.650205e-03,1.786183e-03,⋯,3.318976e-03,1.923151e-03,1.186729e-02,6.802194e-03,2.693582e-03,9.480355e-03,4.084225e-02,3.802441e-03,1.176863e-02,3.169868e-03


species <- rownames(otu)
lapply(species, function(name) {
    otu %>% t() %>% 
    as.data.frame() %>%
    ggplot(aes_string(name)) +
        geom_density()
})

ERROR: Error in princomp.default(otu.rel): 'princomp' can only be used with more units than variables


In [71]:
meta <- meta %>% as.data.frame() 
test <- 
    otu.rel %>% t() %>% as.data.frame() %>% rownames_to_column("sample") %>%left_join(meta, by = "sample") %>%
    mutate(bmi_group = ifelse(bmi_group == "obese", 1, 0)) %>%
        brm(family = bernoulli(link = "logit"), data = ., bmi_group ~ Bifidobacterium*Enterococcus*Bilophila_et_rel)

Compiling the C++ model
Start sampling



SAMPLING FOR MODEL 'bdff27d76867b888fd9852c6a8999a59' NOW (CHAIN 1).

Gradient evaluation took 8.9e-05 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.89 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  400 / 2000 [ 20%]  (Warmup)
Iteration:  600 / 2000 [ 30%]  (Warmup)
Iteration:  800 / 2000 [ 40%]  (Warmup)
Iteration: 1000 / 2000 [ 50%]  (Warmup)
Iteration: 1001 / 2000 [ 50%]  (Sampling)
Iteration: 1200 / 2000 [ 60%]  (Sampling)
Iteration: 1400 / 2000 [ 70%]  (Sampling)
Iteration: 1600 / 2000 [ 80%]  (Sampling)
Iteration: 1800 / 2000 [ 90%]  (Sampling)
Iteration: 2000 / 2000 [100%]  (Sampling)

 Elapsed Time: 7.4957 seconds (Warm-up)
               4.64011 seconds (Sampling)
               12.1358 seconds (Total)


SAMPLING FOR MODEL 'bdff27d76867b888fd9852c6a8999a59' NOW (CHAIN 2).

Gradient evaluation took 3.2e-05 seconds
1000 transitions using 10 leapfrog steps

Warning message:
“There were 1 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
http://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”Warning message:
“Examine the pairs() plot to diagnose sampling problems
”

In [82]:
test %>% tidy()

term,estimate,std.error,lower,upper
b_Intercept,1.039134e+00,6.088009e-01,3.951827e-02,2.048039e+00
b_Bifidobacterium,1.667691e+01,6.560074e+01,-8.698383e+01,1.277616e+02
b_Enterococcus,-2.522936e+03,1.394449e+03,-5.035843e+03,-4.854910e+02
b_Bilophila_et_rel,-9.788658e+03,4.396207e+03,-1.714358e+04,-2.482221e+03
b_Bifidobacterium:Enterococcus,1.511735e+05,9.616729e+04,6.620410e+03,3.274407e+05
b_Bifidobacterium:Bilophila_et_rel,-2.395367e+04,4.826349e+05,-8.483991e+05,7.565450e+05
b_Enterococcus:Bilophila_et_rel,1.412381e+07,9.010731e+06,-1.066422e+05,2.922386e+07
b_Bifidobacterium:Enterococcus:Bilophila_et_rel,-1.147614e+09,7.290539e+08,-2.472195e+09,-6.241767e+07
lp__,-1.492916e+02,2.116272e+00,-1.532722e+02,-1.464778e+02
